<center> <img src="img/UMD-Wordmark.jpg" alt="Alt text" width="40%"> </center>

### <center>Sustainability Center Training Course</center>

## Welcome (Update)

<font color = 'red'>*Add Explanation on how to use course. Can be video or text*

## Example of MBD/MBSE

In [1]:
import ipywidgets as widgets
from IPython.display import display, HTML
from PIL import Image
from fpdf import FPDF
from base64 import b64encode
import os
import io
import json  # PyMuPDF
# import myFunctions as myFn


# Create functions to be used later
def setYpos(currentSection, lastYpos, currentYpos, is_LR):
    lastYpos[is_LR] = currentYpos
    return lastYpos

def printPositionDebug(pos_debug, lastYpos, currentYpos, curretSection, sectionName, is_LR):
    LR_def = ["Left", "Right"]

    if pos_debug == 1:
        print("")
        print(f"🏃 Position Debug -- Section: {sectionName}")
        print(f"Current Section: {curretSection}")
        print(f"Current Side: {LR_def[is_LR]}")
        print(f"End Y Position: {currentYpos}")
        print(f"Last Y Position\n   Left: {lastYpos[0]}  |  Right: {lastYpos[1]}")
        print("")

def list_mbd_subfolders(parent_folder):
    """
    Lists all first-layer subfolders in the specified directory that have '.mbd' in their names.

    Args:
        parent_folder (str): Path to the parent folder.

    Returns:
        list: A list of subfolder paths with '.mbd' in their names.
    """
    if not os.path.isdir(parent_folder):
        print(f"The provided path '{parent_folder}' is not a valid directory.")
        return []

    # Get all first-layer subdirectories
    subfolders = [
        # os.path.join(parent_folder, subfolder)
        subfolder
        for subfolder in os.listdir(parent_folder)
        if os.path.isdir(os.path.join(parent_folder, subfolder)) and '.mbd' in subfolder
    ]

    return subfolders

# Set File Locations
MBD_Location = "example_resources/MBD_Files"
mbdFileList = list_mbd_subfolders(MBD_Location)

# Create a text box
text_box = widgets.Textarea(
    value='',
    placeholder='Type notes...',
    description='Notes:',
    disabled=False
)

# Variable to store the resulting text
resulting_text = ''

def update_text(change):
    global resulting_text
    resulting_text = change['new']
    # print(f"Updated Text: {resulting_text}")

# Attach the update function to the text box
text_box.observe(update_text, names='value')

# Create a dropdown menu
dropdown = widgets.Dropdown(
    options=mbdFileList,
    value=mbdFileList[0],
    description='MBD File:',
    disabled=False
)

# Variable to store the selected dropdown value
selected_option = mbdFileList[0]
def update_selection(change):
    global selected_option    
    selected_option = change['new']
    # print(f"Option selected: {selected_option}")

# Attach the update function to the dropdown
dropdown.observe(update_selection, names='value')

# Display the widget
display(dropdown, text_box)

Dropdown(description='MBD File:', options=('mBlock.mbd', 'iceCleat.mbd'), value='mBlock.mbd')

Textarea(value='', description='Notes:', placeholder='Type notes...')

In [40]:
# A4 Dimentions
w = 210
h = 297

notes = resulting_text
outputFile = selected_option.replace(".mbd", ".pdf")

# partFile = "mBlock.mbd"
# partFile = "iceCleat.mbd"

partFile = MBD_Location + "/" + selected_option

# Import MBD Data
with open(f"{partFile}/info.json", "r") as file:
    partData = json.load(file)

# Setup Document Vairables
docFont = "dejavu-sans"
verticalSectionSpacing = 10
cellWidth = (w-20)/2-10
cellHeight = 8
smallSpacerHeight = 3

xPos = (10, (w-20)/2+20)
lastYpos = (0,0)

currentSection = 0

try:
    partName = partData["general_properties"]["name"]
    partID = partData["general_properties"]["part_id"]
except:
    print(f"🛑 Part Name or Part ID Missing 🛑")

header_start_page = 1
footer_start_page = 0

# Debug Enables
border_enable = False
pos_debug = False
column_choice_debug = False

output_real_file = False

# Setup FPDF Objects
class FPDF(FPDF):
    
    # Header
    def header(self):
        pageNum = self.page_no()
        if pageNum > header_start_page:
            self.set_font(docFont, "", 10)
            self.cell(0,5,f"{partID} [{partName}]", border=border_enable, align="R")
            self.image("example_resources/Images/UM-Dearborn_Logo-horizontal-RGB.png", h=5, x=10)
            self.set_xy(10,30)
            
    # Footer
    def footer(self):
        pageNum = self.page_no()
        footer_height = 5
        footer_font_size = 10

        if pageNum > footer_start_page:
            self.set_font(docFont, "", footer_font_size)
            self.set_y(-10-footer_height)
            self.cell((w-20)/3, footer_height,f"Page: {str(pageNum-footer_start_page)}", border=border_enable)
            self.cell((w-20)/3, footer_height,f"{partID} [{partName}]", border=border_enable, align="C")
            self.cell((w-20)/3, footer_height,"Data Sheet", border=border_enable, align="R")
    
    # Section Title
    def section_title(self, text, spacer_below_height):
        self.set_fill_color(0, 39, 76)
        self.set_font(docFont, "", 14)
        global is_LR

        # Determine if new cell should be left or right
        if lastYpos[0] == lastYpos[1]:
            self.set_xy(xPos[currentSection % 2], lastYpos[currentSection % 2])
            is_LR = currentSection % 2

            # Debug for column choice
            if column_choice_debug == True:
                print("")
                print(f'💈 Column Choice Debug -- Section: {text}')
                print(f'Columns are Identical in Length Chose: {currentSection % 2} out of {lastYpos}')
                print("")
        else:
            shortestColumn = lastYpos.index(min(lastYpos))
            self.set_xy(xPos[shortestColumn], lastYpos[shortestColumn])
            is_LR = shortestColumn

            # Debug for column choice
            if column_choice_debug == True:
                print("")
                print(f'💈 Column Choice Debug -- Section: {text}')
                print(f'Column {shortestColumn} is the shortest at {lastYpos[shortestColumn]} out of {lastYpos}')
                print("")
        
        # Set Font Settings for Section Title
        self.set_text_color(255, 202, 5)
        self.cell(cellWidth, cellHeight, "", border=border_enable, new_x="LEFT", new_y="NEXT")
        self.cell(cellWidth, cellHeight, text, fill=True, align="C", border=border_enable, new_x="LEFT", new_y="NEXT")
        
        # Spacer below section title
        self.cell(cellWidth, spacer_below_height, "", border=border_enable, new_x="LEFT", new_y="NEXT")
        
        # Reset Font for normal text
        self.set_text_color(0, 0, 0)
        self.set_font(docFont, "", 12)

    # Empty Section Title (For images and other non text based sections)
    def no_section_title(self):
        global is_LR
        self.set_fill_color(0, 39, 76)
        self.set_text_color(255, 202, 5)
        
        # Determine if new cell should be left or right
        if lastYpos[0] == lastYpos[1]:
            self.set_xy(xPos[currentSection % 2], lastYpos[currentSection % 2])
            is_LR = currentSection % 2

            # Debug for column choice
            if column_choice_debug == True:
                print("")
                print("💈 Column Choice Debug -- Titleless Section")
                print(f'Columns are Identical in Length Chose: {currentSection % 2} out of {lastYpos}')
                print("")
        else:
            shortestColumn = lastYpos.index(min(lastYpos))
            self.set_xy(xPos[shortestColumn], lastYpos[shortestColumn])
            is_LR = shortestColumn

            # Debug for column choice
            if column_choice_debug == True:
                print("")
                print("💈 Column Choice Debug -- Titleless Section")
                print(f'Column {shortestColumn} is the shortest at {lastYpos[shortestColumn]} out of {lastYpos}')
                print("")
            
        self.cell(cellWidth, cellHeight, "", border=border_enable, new_x="LEFT", new_y="NEXT") # Blank Cell

# Enable FPDF
pdf = FPDF("P", "mm", format=(w,h))

# Setup Fonts and Fallback Fonts
pdf.add_font(fname="example_resources/Fonts/TwitterColorEmoji-SVGinOT.ttf")
pdf.add_font(family="dejavu-sans", style="", fname="example_resources/Fonts/DejaVuSans.ttf")
pdf.add_font(family="dejavu-sans", style="b", fname="example_resources/Fonts/DejaVuSans-Bold.ttf")
pdf.set_fallback_fonts(["TwitterColorEmoji-SVGinOT", "dejavu-sans"])

pdf.set_font(docFont, "", 16)

## Page 1
pdf.add_page()

## Title
title_string = f"{partID} [{partName}]"
title_font_size = 25
pdf.set_text_color(0, 39, 76)
pdf.set_font(docFont, "b", title_font_size)

# Check to see how long title is and change font size accordingly
if pdf.get_string_width(title_string) <= 140:
    pdf.set_x(10)
elif pdf.get_string_width(title_string) >= (w-40):
    pdf.set_x(30)
    title_length = pdf.get_string_width(title_string)
    # Find new font size to make sure it will fit in title block
    while title_length >= (w-40):
        title_font_size += -1
        pdf.set_font(docFont, "b", title_font_size)
        title_length = pdf.get_string_width(title_string)
        if title_font_size <= 12.5:
            break
else:
    pdf.set_x(30)
    
pdf.multi_cell(0, 25/2, title_string, border=border_enable, align="C", max_line_height=title_font_size/2)

## Subtitle
pdf.set_font(docFont, "", 18)
pdf.set_y(21)
pdf.cell(0,9,"Data Sheet", border=border_enable, align="C", new_x="LEFT", new_y="NEXT")
# UMDearborn Logo
image = "example_resources/Images/UM-Dearborn_Logo-RGB.png"
pdf.image(image, h=20 , x=10, y=10)

lastYpos = [pdf.get_y(),pdf.get_y()]

pdf.set_text_color(0, 0, 0)
pdf.set_x(10)

## Sedctions

# Applications Secion
try:
    partApplications = partData["general_properties"]["applications"]
except:
    print("⚠️ general_properties: appliacations not found, skipping ⚠️")
else:
    pdf.section_title("Applications", smallSpacerHeight)

    text = ""

    for i in partApplications:
        if i == partApplications[0]:
            text = text + "  -  " + i
        else:
            text = text + "\n" + "  -  " + i  
        
    pdf.multi_cell(cellWidth, 6, text, border=border_enable, new_x="LEFT", new_y="NEXT")
    
    printPositionDebug(pos_debug, lastYpos, pdf.get_y(), currentSection, "Applications", is_LR)
    lastYpos = setYpos(currentSection, lastYpos, pdf.get_y(), is_LR)
    currentSection += 1

# Image
try:
    partImage = partData["general_properties"]["image"]
except:
    print("⚠️ general_properties: image not found, skipping ⚠️")
else:
    pdf.no_section_title()

    pdf.image(f"{partFile}/{partImage}", w=(w-20)/2-30, x = xPos[currentSection % 2]+10)

    printPositionDebug(pos_debug, lastYpos, pdf.get_y(), currentSection, "Image", is_LR)
    lastYpos = setYpos(currentSection, lastYpos, pdf.get_y(), is_LR)
    currentSection += 1

# Description
try:
    partDescription = partData["general_properties"]["description"]
except:
    print("⚠️ gneral_properties: description not found, skipping ⚠️")
else:
    pdf.section_title("Description", smallSpacerHeight)
    
    pdf.multi_cell(cellWidth, 5, partDescription, align="J", border=border_enable)

    printPositionDebug(pos_debug, lastYpos, pdf.get_y(), currentSection, "Description", is_LR)
    lastYpos = setYpos(currentSection, lastYpos, pdf.get_y(), is_LR)
    currentSection += 1

# Physical Properties
try:
    partPhysicalProperties = partData["physical_properties"]
except:
    print("⚠️ physical_properties not found, skipping ⚠️")
else:
    pdf.section_title("Physical Properties", smallSpacerHeight)

    # Material
    try:
        partMaterial = partPhysicalProperties["material"]
    except:
        print("⚠️ Part Material Missing ⚠️")
    else:
        pdf.cell(cellWidth, 5, f"Material = {partMaterial}", border=border_enable, new_x="LEFT", new_y="NEXT")

    # Mass
    try:
        partMass = partPhysicalProperties["mass"]
        partMassUnit = partPhysicalProperties["mass_unit"]
    except:
        print("⚠️ Part Mass or Mass Unit Missing ⚠️")
    else:
        pdf.cell(cellWidth, 5, f"Mass = {partMass} [{partMassUnit}]", border=border_enable, new_x="LEFT", new_y="NEXT")

    # Volume
    try:
        partVolume = partPhysicalProperties["volume"]
        partVolumeUnit = partPhysicalProperties["volume_unit"]
    except:
        print("⚠️ Part Volume or Volume Unit Missing ⚠️")
    else:
        pdf.cell(cellWidth, 5, f"Volume = {partVolume} [{partVolumeUnit}]", border=border_enable, new_x="LEFT", new_y="NEXT")

    # Hardness
    try:
        partHardness = partPhysicalProperties["hardness"]
        partHardnessUnit = partPhysicalProperties["hardness_unit"]
    except:
        print("⚠️ Part Hardness or Hardness Unit Missing ⚠️")
    else:
        pdf.cell(cellWidth, 5, f"Hardness = {partHardness} [{partHardnessUnit}]", border=border_enable, new_x="LEFT", new_y="NEXT")
    
    printPositionDebug(pos_debug, lastYpos, pdf.get_y(), currentSection, "Physical Properties", is_LR)
    lastYpos = setYpos(currentSection, lastYpos, pdf.get_y(), is_LR)
    currentSection += 1


# Manufacturing Properties
try:
    partManufacturingProperties = partData["manufacturing_properties"]
except:
    print("⚠️ manufacturing_properties not found, skipping ⚠️")
else:
    pdf.section_title("Maufacturing Properties", smallSpacerHeight)

    # Method
    try:
        manufacturingMethod = partManufacturingProperties["method"]
    except:
        print("⚠️ Manufacturing Method Missing ⚠️")
    else:
        pdf.cell(cellWidth, 5, f"Process = {manufacturingMethod}", border=border_enable, new_x="LEFT", new_y="NEXT")

    # Waste Percent
    try:
        manufacturingWaste = partManufacturingProperties["waste_percent"]
    except:
        print("⚠️ Manufacturing Waste Percent Missing ⚠️")
    else:
        pdf.cell(cellWidth, 5, f"Waste = {manufacturingWaste}", border=border_enable, new_x="LEFT", new_y="NEXT")

    # Scrap Rate
    try:
        manufacturingScrap = partManufacturingProperties["scrap_rate"]
    except:
        print("⚠️ Manufacturing Scrap Rate Missing ⚠️")
    else:
        pdf.cell(cellWidth, 5, f"Scrap = {manufacturingScrap}", border=border_enable, new_x="LEFT", new_y="NEXT")

    # Hardness
    try:
        manufacturingCarbonFootprint = partManufacturingProperties["carbon_footprint"]
        manufacturingCarbonFootprintUnit = partManufacturingProperties["carbon_footprint_unit"]
    except:
        print("⚠️ Carborn Footprint or Carbon Footprint Unit Missing ⚠️")
    else:
        pdf.cell(cellWidth, 5, f"Carbon Footprint = {manufacturingCarbonFootprint} [{manufacturingCarbonFootprintUnit}]", border=border_enable, new_x="LEFT", new_y="NEXT")
   
    printPositionDebug(pos_debug, lastYpos, pdf.get_y(), currentSection, "Maunfacturing Properties", is_LR)
    lastYpos = setYpos(currentSection, lastYpos, pdf.get_y(), is_LR)
    currentSection += 1


# Notes
try:
    notes = notes
except:
    print("😡 You messed with the code 😡\n😡 The notes are missing 😡")
else:
    pdf.section_title("Notes", smallSpacerHeight)
    pdf.multi_cell(cellWidth, 5, notes, border=border_enable, align="L")

    printPositionDebug(pos_debug, lastYpos, pdf.get_y(), currentSection, "Notes", is_LR)
    lastYpos = setYpos(currentSection, lastYpos, pdf.get_y(), is_LR)
    currentSection += 1

# # Add Second Page
# pdf.add_page()

# Output PDF file and write file to variable for display
pdf_bytes = pdf.output()
print(f"✅ PDF Generation Successful")

if output_real_file == True:
    pdf.output(outputFile)
    print(f"✅ Output Location: './{outputFile}'")

# New PDF Display
DisplayWidth, DisplayHeight = 800, 800
base64_pdf = b64encode(pdf_bytes).decode("utf-8")
display(HTML(f'<embed height="{DisplayHeight}" src="data:application/pdf;base64,{base64_pdf}" type="application/pdf" width="{DisplayWidth}"/>'))

# # Old PDF Display
# pdf_path = outputFile  # Replace with your PDF file path
# viewer = myFn.PDFViewer(pdf_path)
# viewer.show()

✅ PDF Generation Successful


## Modules

### <font color = '#B8860B'>1. Yellow Belt Level</font>

- **[Module 1 - Ayoub](Modules/1.%20Yellow%20Belt%20Level/Module1.ipynb) - Data** - Understand what data is and why it’s important in digital engineering.  
(*Introduction to basic data types (e.g., integer, string, Boolean), data collection methods, and basic data usage in engineering contexts.*)

- **[Module 2 - Mohanty](Modules/1.%20Yellow%20Belt%20Level/Module2.ipynb) - Model Based Definition (MBD)** - Familiarize with model-based definitions and their purpose in engineering.  
(*Overview of what a model-based definition is, including basic terms and concepts like 3D CAD models, annotations, and tolerances.*)

- **[Module 3 - Chehade](Modules/1.%20Yellow%20Belt%20Level/Module3.ipynb) - Model Based Enterprise (MBE)** - Understand what MBE is and why it’s valuable.  
(*Introduction to MBE concepts and benefits, such as data integration, digital thread, and lifecycle management.*)

- **[Module 4 - Ayoub](Modules/1.%20Yellow%20Belt%20Level/Module4.ipynb) - Modeling and Simulation** - Understand the basics of modeling and simulation.  
(*Overview of modeling types (e.g., static vs. dynamic models), simple simulations, and their purposes in engineering.*)

- **[Module 5 - Pannier](Modules/1.%20Yellow%20Belt%20Level/Module5.ipynb) - Model Based Process Control** - Understand how process data can be collected and used to improve performance.  
(*Introduction to process data collection methods, such as sensors.*)

- **[Module 6 - Hu](Modules/1.%20Yellow%20Belt%20Level/Module6.ipynb) - Digital Twins** - Understand what a digital twin is and the concept’s basics.  
(*Define digital twins, common applications, and how they relate to physical assets in engineering.*)

- **[Module 7 - All - Ask Dr CHehade for Engine Monitoring Data](Modules/1.%20Yellow%20Belt%20Level/Module7.ipynb) - Digital Engineering Practical Examples** - Identify the foundational elements of digital engineering.  
(*Introduction to digital engineering principles, such as CAD, PLM (Product Lifecycle Management), basic software used in the field, and the basics of process monitoring.*)

- **[Module 8 - All, refer to NFS proposal and derive contets from there](Modules/1.%20Yellow%20Belt%20Level/Module8.ipynb) - Digital Engineering Challenges** - Equip students with problem-solving techniques for basic digital engineering challenges.  
(*Address common issues in digital engineering, such as data compatibility and communication challenges.*)

### <font color = '#006400'>2. Green Belt Level</font>

- **[Module 1](Modules/2.%20Green%20Belt%20Level/Module1.ipynb) - Data** - Recognize and use different data types for engineering applications.  
(*Study data structures (e.g., lists, dictionaries), data formatting (CSV, JSON), and simple data analysis using tools like Excel or basic scripts.*)

- **[Module 2](Modules/2.%20Green%20Belt%20Level/Module2.ipynb) - Model Based Definition (MBD)** - Interact with and use MBD in simple engineering tasks.  
(*Hands-on experience with MBD systems, learning to interpret and use CAD annotations, dimensions, and simple assembly views for engineering components.*)

- **[Module 3](Modules/2.%20Green%20Belt%20Level/Module3.ipynb) - Model Based Enterprise (MBE)** - Learn how MBE integrates with various business practices.  
(*Exploration of MBE workflows in engineering, focusing on collaborative tools and MBE’s role in business operations like supply chain and quality management.*)

- **[Module 4](Modules/2.%20Green%20Belt%20Level/Module4.ipynb) - Modeling and Simulation** - Explore different modeling tools and perform basic simulations.  
(*Hands-on activities with common modeling tools, running simple simulations, and interpreting results for engineering decision-making.*)

- **[Module 5](Modules/2.%20Green%20Belt%20Level/Module5.ipynb) - Model Based Process Control** - Explore how process modeling can lead to process improvements.  
(*Use simple process models to analyze data trends, and introduce predictive maintenance concepts using basic data analysis tools.*)

- **[Module 6](Modules/2.%20Green%20Belt%20Level/Module6.ipynb) - Digital Twins** - Evaluate the utility of a digital twin in specific scenarios.  
(*Study and interact with basic digital twin examples, understanding how to create, monitor, and use them for simple engineering systems.*)

- **[Module 7](Modules/2.%20Green%20Belt%20Level/Module7.ipynb) - Digital Engineering Practice** - Apply digital engineering principles to solve practical problems.  
(*Learn about data management, version control, and workflow optimization using practical tools and simple software integration.*)

- **[Module 8](Modules/2.%20Green%20Belt%20Level/Module8.ipynb) - Digital Engineering Challenges** - Provide strategies to solve intermediate-level digital engineering problems.  
(*Troubleshoot digital engineering workflows, such as data sharing issues, software incompatibility, and basic cybersecurity concerns.*)

### <font color = '#000000'>3. Black Belt Level</font>

- **[Module 1](Modules/3.%20Black%20Belt%20Level/Module1.ipynb) - Data** - Apply data processing and analysis to solve engineering problems.  
(*Introduction to data processing and analysis using Python (pandas library, data cleaning, basic statistics) to extract insights from engineering datasets.*)

- **[Module 2](Modules/3.%20Black%20Belt%20Level/Module2.ipynb) - Model Based Definition (MBD)** - Integrate MBD into complex engineering workflows.  
(*Deep dive into advanced MBD functionalities, creating model-based definitions, and using MBD for detailed analysis and collaborative workflows.*)

- **[Module 3](Modules/3.%20Black%20Belt%20Level/Module3.ipynb) - Model Based Enterprise (MBE)** - Drive MBE adoption within complex organizational settings.  
(*Analyze case studies on MBE implementation, strategies for MBE integration in large enterprises, and potential business benefits such as reducing cycle times and improving data transparency.*)

- **[Module 4](Modules/3.%20Black%20Belt%20Level/Module4.ipynb) - Modeling and Simulation** - Perform complex simulations to drive engineering decisions.  
(*Advanced simulation techniques (e.g., finite element analysis), evaluating simulation results, and applying simulations in real-world engineering scenarios.*)

- **[Module 5](Modules/3.%20Black%20Belt%20Level/Module5.ipynb) - Model Based Process Control** - Design and optimize process control systems using advanced data models.  
(*Develop and implement process models to optimize performance, including advanced techniques like real-time monitoring, predictive analytics, and feedback loops.*)

- **[Module 6](Modules/3.%20Black%20Belt%20Level/Module6.ipynb) - Digital Twins** - Develop and deploy digital twins for complex systems.  
(*Explore advanced digital twin concepts, such as real-time data integration, IoT sensor data, and lifecycle management, and assess their impact on engineering projects.*)

- **[Module 7](Modules/3.%20Black%20Belt%20Level/Module7.ipynb) - Digital Engineering Practice** - Lead digital engineering practices in an organizational setting.  
(*Advanced use of digital engineering platforms, designing complex workflows, integrating cross-departmental digital engineering practices, and evaluating outcomes.*)

- **[Module 8](Modules/3.%20Black%20Belt%20Level/Module8.ipynb) - Digital Engineering Challenges** - Develop solutions for complex and emerging digital engineering challenges.  
(*Address advanced challenges like scalability, integration with legacy systems, and strategies for digital transformation, including managing data security and governance.*)
